Dans cette partie, nous mettons en place un modèle de random forest univarié : en nous limitant aux titres ne comportant qu'un seul label, nous cherchons à prédire au mieux ce label.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#importation des données
df0 = pd.read_parquet("C:/Users/22sir/Desktop/Stat app/dataset2.parquet", engine="pyarrow")
df = df0.copy()
df["Nombre labels"] = df.iloc[:,0:21].sum(axis=1)
df.sort_values("Nombre labels")

#dataframe des labels
labels = df.columns[:21]
df_labels = pd.DataFrame(data=labels,columns=['Label'])
print(labels)

C:\Users\22sir\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Index(['asian', 'rnb', 'reggae', 'blues', 'pop', 'dance', 'folk',
       'arabic-music', 'indie', 'rock', 'soulfunk', 'latin', 'classical',
       'k-pop', 'brazilian', 'metal', 'rap', 'jazz', 'electronic', 'african',
       'country'],
      dtype='object')


In [2]:
#juste 1000 titres pour limiter temps de calcul
n=df.shape[0]
p=0.1
df_audio2 = df.iloc[:,:24].drop("song_index",axis="columns")
df_audio = df_audio2.sample(int(p*n), random_state=0)
df_audio

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,k-pop,brazilian,metal,rap,jazz,electronic,african,country,audio_features,usage_features
22511,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,"[0.16615089774131775, -0.2734835743904114, -0....","[0.01044429975649563, 0.004877081975633674, -0..."
30731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"[-0.21683786809444427, 0.4627639651298523, -0....","[-0.000511197983065257, -0.0012838325543555339..."
14786,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0.11202938854694366, -0.0963234156370163, 0.1...","[0.01235335155680325, -0.013374900962778598, -..."
25482,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[-0.10709291696548462, -0.025860929861664772, ...","[0.004814619140054636, 0.0010666397175422878, ..."
18437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,"[-0.21349938213825226, -0.26145464181900024, -...","[0.00629359601744014, -0.005044553867404549, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[0.13672180473804474, -0.16638535261154175, 0....","[0.002337388929889797, 0.005100165738625514, -..."
20081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"[-0.11083365231752396, 0.08312846720218658, 0....","[0.0029368403258820393, 0.01545390357139899, -..."
21180,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,"[0.25639811158180237, 0.16245372593402863, 1.2...","[-0.0029479198451235547, 0.002515210300079332,..."
20886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,"[-0.1506918966770172, 0.27602189779281616, 0.4...","[-0.0024020758090555166, -0.000236174332262478..."


In [3]:
#décomposition des array des audio features en colonnes pour faciliter la manipulation
m = len(df_audio.iloc[0,21])
for i in range(m):
    df_audio["Audio "+str(i)]= 0
for i in range(len(df_audio.index)):
    for j in range(m):
        df_audio.iloc[i,j+23]= df_audio.iloc[i,21][j]
df_audio

C:\Users\22sir\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,Audio 246,Audio 247,Audio 248,Audio 249,Audio 250,Audio 251,Audio 252,Audio 253,Audio 254,Audio 255
22511,0,0,0,0,0,0,0,0,1,1,...,-0.102349,0.145390,0.294675,-0.174709,-0.616194,-0.072464,0.124256,-0.104643,0.600556,-0.620525
30731,0,0,0,0,0,0,0,0,0,0,...,0.633235,0.137259,-0.233275,-0.016479,-0.412676,1.847315,-0.051340,0.535907,0.478039,-0.205460
14786,0,0,0,0,1,0,0,0,0,0,...,0.015761,0.242299,-0.138685,-0.109917,-0.440789,0.146961,-0.066796,-0.191200,0.590519,-0.573529
25482,0,1,0,0,0,0,0,0,0,0,...,-0.041033,0.051313,-0.165650,-0.109368,-0.238462,0.837006,-0.312524,0.181177,0.185863,-0.387283
18437,0,0,0,0,0,0,0,0,0,0,...,0.775451,0.581091,-0.229473,-0.199970,-0.583064,1.224286,0.044158,-0.190305,2.084058,-0.620527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31820,0,0,0,0,0,0,0,0,0,0,...,-0.003919,0.081448,-0.133762,0.115193,-0.327597,0.147361,-0.145301,0.056444,0.032732,-0.415448
20081,0,0,0,0,0,0,0,0,0,0,...,0.169941,0.005655,-0.212875,-0.060785,-0.294083,0.830844,-0.555644,0.153756,0.570822,-0.003506
21180,0,0,0,0,0,0,0,0,0,0,...,1.242553,0.852992,0.381011,-0.205893,-0.362584,-0.004962,-0.848327,0.020741,2.207142,-0.559233
20886,0,0,0,0,0,0,0,0,0,0,...,-0.009067,0.055593,-0.212452,0.198902,-0.198440,1.482617,-0.407542,0.600713,-0.152999,0.105779


In [4]:
# importation des modules du random forest
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.externals import joblib
from sklearn import metrics

C:\Users\22sir\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\22sir\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

Le random forest

In [5]:
print(labels)

Index(['asian', 'rnb', 'reggae', 'blues', 'pop', 'dance', 'folk',
       'arabic-music', 'indie', 'rock', 'soulfunk', 'latin', 'classical',
       'k-pop', 'brazilian', 'metal', 'rap', 'jazz', 'electronic', 'african',
       'country'],
      dtype='object')


In [6]:
#randomforest sur chacun des labels et calcul de l'accuracy
Accuracy=[0]*21
df_hat=pd.DataFrame()
df_test=pd.DataFrame()
n=21
auc_labels=[]
i=0
for labels in labels:
    #df_genre = df_audio.iloc[:,[j]+[i for i in range(22,278)]]
    Y = df_audio.iloc[:,i]
    X = df_audio.iloc[:,23:257]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
    
    filename= '../'+labels
    
#Create a Gaussian Classifier
    clf=RandomForestClassifier(n_estimators=100)
    
#Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,Y_train)
    Y_hat=clf.predict(X_test)
    df_hat[labels] = Y_hat
    df_test[labels]=Y_test
    print(df_hat.head(100))
    plt.show
    #plot_roc_curve(y_test,y_pred,"hola",save=False,name_fig=None)
    Accuracy[i] = metrics.accuracy_score(Y_test, Y_hat)
    i+=1

C:\Users\22sir\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
C:\Users\22sir\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curr

    asian
0       0
1       0
2       0
3       0
4       0
..    ...
95      0
96      0
97      0
98      0
99      0

[100 rows x 1 columns]


AttributeError: module 'sklearn.metrics' has no attribute 'plot_roc_curve'

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
df_labels["Accuracy"] = Accuracy
df_labels